In [309]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
pagerankによるp(word|topic)の計算
purityによるスコアの計算
p(word|topic)から凸２次計画問題を解くことで、p(topic)を求める
エッジの中でインエッジとアウトエッジの数をノードごとに羅列していく
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP
import copy
import re

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}
    
    
# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        pattern = "_[0-9]+"
        word1 = re.sub(pattern, "", word)
        dict_cluster[cluster].append(word1)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["old_edge"], dict_network["old_weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["old_vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["old_vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

In [359]:
# クラスタリング後に、ノードごとにどのクラスタにエッジを持っているのか計算
def cal_cluster_to_edge(dict_network, low_fleq=50, low_rate=0.8):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)
        
    # wordをclusterに変換するための辞書を作成する
    dict_word_to_cluster = {}
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_word_to_cluster[word] = cluster
        
    # wordをidに変換するための辞書を作成する
    dict_word_to_id = {}
    for i, word in enumerate(dict_network["vertex"]):
        dict_word_to_id[word] = i
    
    # id_to_idのマトリックス
    matrix_id_to_cluster = np.zeros((len(dict_word_to_id), len(dict_cluster)))
    for row, weight in zip(dict_network["edge"], dict_network["weight"]):
        matrix_id_to_cluster[dict_word_to_id[row[0]]][dict_word_to_cluster[row[1]]] += weight
        matrix_id_to_cluster[dict_word_to_id[row[1]]][dict_word_to_cluster[row[0]]] += weight
    
    # 指定したハイパーパラメータよりも高い値を記録した単語を所属クラスタとともに辞書に登録する
    dict_word_to_list_cluster = {}
    for cluster, word, row in zip(dict_network["cluster"], dict_network["vertex"], matrix_id_to_cluster):
        top_num = max(row)
        if np.sum(row) >= low_fleq and len(np.where( row/top_num >= low_rate)[0])>=2:
            print word
            dict_word_to_list_cluster[word] = np.where(row/top_num >= low_rate)[0]
    
    
    # 新しく分裂するノードクラスターを元のクラスターに記録
    for word, row in dict_word_to_list_cluster.items():
        for num in row:
            # dict_word_to_cluster[word+"_"+str(num)] = num
            dict_word_to_cluster[word+"_"+str(num)] = dict_word_to_cluster[word]
    
    dict_network["old_edge"] = copy.deepcopy(dict_network["edge"])
    dict_network["old_weight"] = copy.deepcopy(dict_network["weight"])
    dict_network["old_vertex"] = copy.deepcopy(dict_network["vertex"])
    # ここの計算が絶対に間違ってる => diffsplitで確認したが大丈夫そう・・・
    for word in dict_word_to_list_cluster.keys():
        list_edge_new = []
        list_weight_new = []
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            if row[0] == word:
                if dict_word_to_cluster[row[1]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0]+"_"+str(dict_word_to_cluster[row[1]]), row[1]])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        list_edge_new.append([row[0]+"_"+str(num), row[1]])
                        list_weight_new.append(float(weight)/len(dict_word_to_list_cluster[word]))
            elif row[1] == word:
                if dict_word_to_cluster[row[0]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0], row[1]+"_"+str(dict_word_to_cluster[row[0]])])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        list_edge_new.append([row[0], row[1]+"_"+str(num)])
                        list_weight_new.append(float(weight)/len(dict_word_to_list_cluster[word]))
            else:
                list_edge_new.append([row[0], row[1]])
                list_weight_new.append(weight)
        else:
            dict_network["edge"] = copy.deepcopy(list_edge_new)
            dict_network["weight"] = copy.deepcopy(list_weight_new)
            list_vertices = list(set([word for row in dict_network["edge"] for word in row]))
            dict_network["vertex"] = list_vertices
    
    return dict_network

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [355]:
# エッジリストの読み込み
list_edge = readcsv("./files/kaigo_honne/list_edgelist20160127.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
# 元のネットワークのpagerankを求める
dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])
# クラスタリングの結果をもとに、どのノードから何本のエッジが出ているか計算する
dict_network_master = cal_cluster_to_edge(dict_network_master, low_fleq=100, low_rate=0.7)
"""
list_tmp = []
for row, weight in zip(dict_network_master["edge"], dict_network_master["weight"]):
    list_tmp.append([row[0], row[1], weight])
list_tmp_old = []
for row, weight in zip(dict_network_master["old_edge"], dict_network_master["old_weight"]):
    list_tmp_old.append([row[0], row[1], weight])
list_tmp = sorted(list_tmp, key=lambda x:x[0])
list_tmp_old = sorted(list_tmp_old, key=lambda x:x[0])
for row in list_tmp:
    print row[0], row[1], row[2]
print "============="
for row in list_tmp_old:
    print row[0], row[1], row[2]
"""
print len(dict_network_master["weight"])
print len(dict_network_master["edge"])
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
print "クラスタ数: ", len(list_dict_network_sub)

# トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
list_dict_prob = []
for i in range(len(list_dict_network_sub)):
    list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
    list_dict_prob.append({row[0]: row[1] for row in list_word_page})
    
# 凸２次計画問題を解いて、p(topic)を求める
list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)

方々
非常
いい
充実
大変
見学
9888
9888
クラスタ数:  9

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -1.6718e-03 -1.0026e+00  1e+00  3e-16  3e+00
 1: -1.6730e-03 -1.2561e-02  1e-02  1e-16  4e-02
 2: -1.7479e-03 -2.5381e-03  8e-04  2e-16  3e-03
 3: -1.9108e-03 -2.0380e-03  1e-04  1e-16  1e-18
 4: -1.9299e-03 -1.9412e-03  1e-05  4e-17  2e-18
 5: -1.9310e-03 -1.9316e-03  6e-07  7e-17  1e-18
 6: -1.9310e-03 -1.9310e-03  9e-09  1e-16  1e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0019310228 (feasible, MaxResidual = 0)


In [356]:
num = 7
list_tmp = sorted(list_dict_prob[num].items(), key=lambda x: x[1], reverse=True)
for i in range(10):
    print list_tmp[i][0], list_tmp[i][1]

ライフ 0.0264298860773
ロング 0.0217871750132
芦屋 0.0027399213852
成城 0.0027399213852
女性 0.0027399213852
低価格 0.0027399213852
ケア 0.0027399213852
有料老人ホーム 0.0027399213852
明るい 0.0027399213852
プラン 0.00271971433085


### 定量評価部分
1. 形態素解析済みセンテンスを読み込み、確率を計算し、どのクラスに属するか計算
2. 予測クラスと実際クラスのリストを作成する

In [357]:
list_sep_words = readdump("./files/kaigo_honne/list_sep_words_label.dump")
list_words= readcsv("./files/kaigo_honne/list_sentence_user_label.csv")
# 確率が最大になるクラスを予想する
# 実質ラベルがintじゃない場合は、エラーとして、記録しない
list_predict_measure = []
list_words_rev = []
error_count = 0
for row, row1 in zip(list_sep_words, list_words):
    try:
        class_tmp = 0
        prob_tmp = 0
        predict_class = []
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row[0]:
                prob_tmp_tmp *= float(dict_prob[word])
            prob_tmp_tmp *= list_prob_topic[num]
            if prob_tmp_tmp > prob_tmp:
                class_tmp = num
                prob_tmp = prob_tmp_tmp
        list_predict_measure.append([class_tmp, row[1]])
        list_words_rev.append(row1)
    except:
        error_count += 1
print "エラーデータ数: ", error_count

エラーデータ数:  68


###計算結果の表示

* 100, 0.7, or、切り捨て、割りあてるクラスタを変更,で計算した場合

In [261]:
cal_f_measure(list_predict_measure)

Purity:  0.495293045855
Inverse Purity:  0.650470695415
F-value:  0.562373551138


* 100, 0.7, or、少数、割りあてるクラスタを変更,で計算した場合

In [358]:
cal_f_measure(list_predict_measure)

Purity:  0.494989371394
Inverse Purity:  0.648041299727
F-value:  0.561268500826


* 50, 0.7, or、切り捨て、割りあてるクラスタを変更,で計算した場合

In [265]:
cal_f_measure(list_predict_measure)

Purity:  0.488004858791
Inverse Purity:  0.646219252961
F-value:  0.556077290232


* 100, 0.6, or、切り捨て、割りあてるクラスタを変更,で計算した場合

In [287]:
cal_f_measure(list_predict_measure)

Purity:  0.493167324628
Inverse Purity:  0.644093531734
F-value:  0.558615698551


* 100, 0.7, or、切り捨てで計算した場合

In [247]:
cal_f_measure(list_predict_measure)

Purity:  0.486486486486
Inverse Purity:  0.649559672032
F-value:  0.556318949262


* 100, 0.9, or, 切り捨てで計算した場合

In [215]:
cal_f_measure(list_predict_measure)

Purity:  0.487093835408
Inverse Purity:  0.623443668387
F-value:  0.546898356081


* 50, 0.8, or, 切り捨てで計算した結果

In [233]:
cal_f_measure(list_predict_measure)

Purity:  0.494078348011
Inverse Purity:  0.638931065897
F-value:  0.557245159054


* 50, 0.7, or, 切り捨てで計算した結果

In [237]:
cal_f_measure(list_predict_measure)

Purity:  0.486486486486
Inverse Purity:  0.649559672032
F-value:  0.556318949262


In [18]:
cal_f_measure(list_predict_measure)

Purity:  0.492256301245
Inverse Purity:  0.639234740358
F-value:  0.556199417134
